In [1]:
!pip install python-dateutil
!pip install pandas
!pip install GitPython

In [3]:
# getting all of the imports out of the way

import pandas as pd
import git
from io import StringIO
from dateutil.parser import parse

In [4]:
# alternative way to get all of the commit information potentially:

GIT_REPO_PATH = r'../repositories/big-bang'

repo = git.Repo(GIT_REPO_PATH, odbt=git.GitCmdObjectDB)

repo

commits = pd.DataFrame(repo.iter_commits('master'), columns=['raw'])
commits.head()

,raw
0,4c2a531567007909429bbe6840f69556c1b83772
1,488f89f27d38579be9fe1f6be677ba3e96b95f73
2,4989e8adf01ecd40016362e424b279c5e9b704d4
3,d8d05afdb8f43af607a30cf55475a35bbe9b65f3
4,11b3b19854f664968dcf58aac56079d123533a96


In [16]:
last_commit = commits.iloc[0]

last_commit['raw'].__slots__

('tree',
 'author',
 'authored_date',
 'author_tz_offset',
 'committer',
 'committed_date',
 'committer_tz_offset',
 'message',
 'parents',
 'encoding',
 'gpgsig')

In [6]:
commits['author'] = commits['raw'].apply(lambda x: x.author.name)
commits['committed_date'] = commits['raw'].apply(lambda x: pd.to_datetime(x.committed_datetime))
commits['message'] = commits['raw'].apply(lambda x: x.message)
commits['parents'] = commits['raw'].apply(lambda x: x.parents)
commits['committer'] = commits['raw'].apply(lambda x: x.committer)

In [7]:
commits.head()

,raw,author,committed_date,message,parents,committer
0,4c2a531567007909429bbe6840f69556c1b83772,Ryan Garcia,2023-03-14 21:49:40+00:00,Merge branch 'enable-mtls-for-neuvector-metric...,"(4989e8adf01ecd40016362e424b279c5e9b704d4, 488...",Ryan Garcia
1,488f89f27d38579be9fe1f6be677ba3e96b95f73,Brett Charrier,2023-03-14 21:49:40+00:00,Enable mTLS for Neuvector metrics\n,"(b7c7731260cb9993c8b437103e5e4f526bf70296,)",Ryan Garcia
2,4989e8adf01ecd40016362e424b279c5e9b704d4,Micah Nagel,2023-03-14 17:52:55+00:00,Merge branch 'update-kyverno-policies-tag-1.1....,"(11b3b19854f664968dcf58aac56079d123533a96, d8d...",Micah Nagel
3,d8d05afdb8f43af607a30cf55475a35bbe9b65f3,mr-bot,2023-03-14 17:52:55+00:00,kyverno-policies update to 1.1.0-bb.3\n,"(ddd6e3eb73cec0bddee1ee224501e2f12cb35b08,)",Micah Nagel
4,11b3b19854f664968dcf58aac56079d123533a96,Micah Nagel,2023-03-14 17:52:32+00:00,Merge branch 'update-kyverno-tag-2.6.5-bb.3' i...,"(15c82732e41ae61799da95dacfda9f826d8bb755, 5de...",Micah Nagel


In [26]:
import random

# understanding stats associated with a random commit

rand_commit = commits.iloc[random.randint(0, commits.shape[0])]['raw']

rand_commit.stats.files

{'chart/values.yaml': {'insertions': 1, 'deletions': 1, 'lines': 2}}

In [29]:
# in order to make a stacked dataframe to later merge, we'll wrap a dataframe around a pandas series of the stats and call the stack() method

rand_commit_df = pd.DataFrame(pd.Series(rand_commit.stats.files)).stack()

In [34]:
stats = pd.DataFrame(commits['raw'].apply(lambda x: pd.Series(x.stats.files, dtype=object)).stack()).reset_index(level=1)
stats = stats.rename(columns={ 'level_1' : 'filename', 0 : 'stats_modifications'})
stats_modifications = stats['stats_modifications'].apply(lambda x: pd.Series(x))
stats = stats.join(stats_modifications)
del(stats['stats_modifications'])

In [35]:
stats.head()

,filename,insertions,deletions,lines
0,chart/templates/neuvector/values.yaml,9,2,11
0,chart/templates/neuvector/values.yaml,1,1,2
0,chart/values.yaml,9,2,11
0,chart/values.yaml,1,1,2
1,chart/templates/neuvector/values.yaml,9,2,11


In [36]:
commits = commits.join(stats)
del(commits['raw'])

commits.head()

,author,committed_date,message,parents,committer,filename,insertions,deletions,lines
0,Ryan Garcia,2023-03-14 21:49:40+00:00,Merge branch 'enable-mtls-for-neuvector-metric...,"(4989e8adf01ecd40016362e424b279c5e9b704d4, 488...",Ryan Garcia,chart/templates/neuvector/values.yaml,9.0,2.0,11.0
0,Ryan Garcia,2023-03-14 21:49:40+00:00,Merge branch 'enable-mtls-for-neuvector-metric...,"(4989e8adf01ecd40016362e424b279c5e9b704d4, 488...",Ryan Garcia,chart/templates/neuvector/values.yaml,1.0,1.0,2.0
0,Ryan Garcia,2023-03-14 21:49:40+00:00,Merge branch 'enable-mtls-for-neuvector-metric...,"(4989e8adf01ecd40016362e424b279c5e9b704d4, 488...",Ryan Garcia,chart/values.yaml,9.0,2.0,11.0
0,Ryan Garcia,2023-03-14 21:49:40+00:00,Merge branch 'enable-mtls-for-neuvector-metric...,"(4989e8adf01ecd40016362e424b279c5e9b704d4, 488...",Ryan Garcia,chart/values.yaml,1.0,1.0,2.0
1,Brett Charrier,2023-03-14 21:49:40+00:00,Enable mTLS for Neuvector metrics\n,"(b7c7731260cb9993c8b437103e5e4f526bf70296,)",Ryan Garcia,chart/templates/neuvector/values.yaml,9.0,2.0,11.0
